In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

colab = True
name = "0070-GAN-0035"

if not colab: 
  train_dir = r'G:\Mi unidad\Uni\Cuarto curso\Primer cuatrimestre\TFG\WebScrapping\train'
  validation_dir = r'G:\Mi unidad\Uni\Cuarto curso\Primer cuatrimestre\TFG\WebScrapping\validation'
  test_dir = r'G:\Mi unidad\Uni\Cuarto curso\Primer cuatrimestre\TFG\WebScrapping\test'

else:
  train_dir = r'/content/drive/MyDrive/train'
  #validation_dir = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/validation'
  test_dir = r'/content/drive/MyDrive/test'
  mini_test_dir = r'/content/drive/MyDrive/mini_test'

batch_size = 1
img_height = 1792
img_width = 1280
img_gen = ImageDataGenerator()

train_gen = img_gen.flow_from_directory(train_dir,target_size=(img_height, img_width), 
                                         batch_size = batch_size, color_mode = "rgb", class_mode = "input")

#validation_gen = img_gen.flow_from_directory(validation_dir,target_size=(img_height, img_width), 
#                                         batch_size = batch_size, color_mode = "rgb", class_mode = "input")

test_gen = img_gen.flow_from_directory(test_dir,target_size=(img_height, img_width), 
                                         batch_size = batch_size, color_mode = "rgb", class_mode = "input")

mini_test_gen = img_gen.flow_from_directory(mini_test_dir,target_size=(img_height, img_width), 
                                         batch_size = 1, color_mode = "rgb", class_mode = "input", shuffle = False)

Found 1574 images belonging to 1 classes.
Found 3 images belonging to 1 classes.
Found 6 images belonging to 1 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.image import rgb_to_grayscale, grayscale_to_rgb

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def to_grayscale_then_rgb(image):
    image = rgb_to_grayscale(image)
    image = grayscale_to_rgb(image)
    return image

def preprocess(data):
    i = 0
    for imgs in data:
        img, label = imgs
        yield (to_grayscale_then_rgb(img))/ 255, label/ 255

In [ ]:
# example of defining a composite model for training the generator model
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from tensorflow.keras.layers import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model

# define the discriminator model
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02, seed = 123)
	# source image input
	in_src_image = Input(shape=image_shape)
	# target image input
	in_target_image = Input(shape=image_shape)
	# concatenate images channel-wise
	merged = Concatenate()([in_src_image, in_target_image])
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)
	# define model
	model = Model([in_src_image, in_target_image], patch_out)
	# compile model
	opt = Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
	return model

# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
	# weight initialization
	init = RandomNormal(stddev=0.02, seed = 1)
	# add downsampling layer
	g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# conditionally add batch normalization
	if batchnorm:
		g = BatchNormalization()(g, training=True)
	# leaky relu activation
	g = LeakyReLU(alpha=0.2)(g)
	return g

# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
	# weight initialization
	init = RandomNormal(stddev=0.02, seed = 2)
	# add upsampling layer
	g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# add batch normalization
	g = BatchNormalization()(g, training=True)
	# conditionally add dropout
	if dropout:
		g = Dropout(0.5)(g, training=True)
	# merge with skip connection
	g = Concatenate()([g, skip_in])
	# relu activation
	g = Activation('relu')(g)
	return g

# define the standalone generator model
def define_generator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02, seed = 5)
	# image input
	in_image = Input(shape=image_shape)
	# encoder model: C64-C128-C256-C512-C512-C512-C512-C512
	e1 = define_encoder_block(in_image, 64, batchnorm=False)
	e2 = define_encoder_block(e1, 128)
	e3 = define_encoder_block(e2, 256)
	e4 = define_encoder_block(e3, 512)
	e5 = define_encoder_block(e4, 512)
	e6 = define_encoder_block(e5, 512)
	e7 = define_encoder_block(e6, 512)
	# bottleneck, no batch norm and relu
	b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
	b = Activation('relu')(b)
	# decoder model: CD512-CD1024-CD1024-C1024-C1024-C512-C256-C128
	d1 = decoder_block(b, e7, 512)
	d2 = decoder_block(d1, e6, 512)
	d3 = decoder_block(d2, e5, 512)
	d4 = decoder_block(d3, e4, 512, dropout=False)
	d5 = decoder_block(d4, e3, 256, dropout=False)
	d6 = decoder_block(d5, e2, 128, dropout=False)
	d7 = decoder_block(d6, e1, 64, dropout=False)
	# output
	g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape):
	# make weights in the discriminator not trainable
	for layer in d_model.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# define the source image
	in_src = Input(shape=image_shape)
	# connect the source image to the generator input
	gen_out = g_model(in_src)
	# connect the source input and generator output to the discriminator input
	dis_out = d_model([in_src, gen_out])
	# src image as input, generated image and classification output
	model = Model(in_src, [dis_out, gen_out])
	# compile model
	opt = Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
	return model

# define image shape
image_shape=(img_height,img_width,3)
# define the models
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape)
# summarize the model
gan_model.summary()
# plot the model
#plot_model(gan_model, to_file='gan_model_plot.png', show_shapes=True, show_layer_names=True)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 1792, 1280,  0           []                               
                                 3)]                                                              
                                                                                                  
 model_1 (Functional)           (None, 1792, 1280,   54429315    ['input_4[0][0]']                
                                3)                                                                
                                                                                                  
 model (Functional)             (None, 112, 80, 1)   6968257     ['input_4[0][0]',                
                                                                  'model_1[0][0]']          

In [ ]:
import numpy as np
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
	#trainA, trainB = dataset
	trainA, trainB = next(dataset)
	# choose random instances
	#ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA, trainB
	# generate 'real' class labels (1)
	y = np.ones((n_samples, patch_shape[0], patch_shape[1], 1))
	return [X1, X2], y

# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = np.zeros((len(X), patch_shape[0], patch_shape[1], 1))
	return X, y

# train pix2pix models
def train(d_model, g_model, gan_model, dataset, validation_dataset, n_steps, n_validation, n_batch, n_patch=16):
	# unpack dataset
	n_patch = (112,80)
	#trainA, trainB = dataset
	# calculate the number of batches per training epoch
	#bat_per_epo = int(len(trainA) / n_batch)
	bat_per_epo = 2000
	# calculate the number of training iterations
	history = {"d_loss1":[],
	           "d_loss2":[],
	           "g_loss":[],
						 "val_d_loss1":[],
						 "val_d_loss2":[],
						 "val_g_loss":[]	}

	history_d_loss1 = []
	history_d_loss2 = []
	history_g_loss = []
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		[X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
		# update discriminator for real samples


		d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
		# update discriminator for generated samples
		d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
		# update the generator
		g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
		# summarize performance
		"""
		history_d_loss1.append(d_loss1)
		history_d_loss2.append(d_loss2)
		history_g_loss.append(g_loss)
		"""	
		print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
		"""	
		if (i+1) % n_validation == 0 :
			history["d_loss1"].append(np.mean(history_d_loss1))
			history["d_loss2"].append(np.mean(history_d_loss2))
			history["g_loss"].append(np.mean(history_g_loss))
			history_d_loss1 = []
			history_d_loss2 = []
			history_g_loss = []

			# select a batch of real samples
			[X_realA, X_realB], y_real = generate_real_samples(validation_dataset, n_batch, n_patch)
			# generate a batch of fake samples
			X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
			# update discriminator for real samples

			d_loss1 = d_model.evaluate([X_realA, X_realB], y_real)
			# update discriminator for generated samples
			d_loss2 = d_model.evaluate([X_realA, X_fakeB], y_fake)
			# update the generator
			g_loss, _, _ = gan_model.evaluate(X_realA, [y_real, X_realB])
			# summarize performance

			history["val_d_loss1"].append(d_loss1)
			history["val_d_loss2"].append(d_loss2)
			history["val_g_loss"].append(g_loss)
			"""
	return history

In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 4,n_steps = 500, n_validation = 20)

1/1 [==============================] - 14s 14s/step
>1, d1[0.573] d2[0.750] g[67.794]
1/1 [==============================] - 0s 124ms/step
>2, d1[0.567] d2[0.722] g[65.585]
1/1 [==============================] - 0s 124ms/step
>3, d1[1.008] d2[0.689] g[57.195]
1/1 [==============================] - 0s 403ms/step
>4, d1[1.019] d2[0.542] g[55.282]
1/1 [==============================] - 0s 127ms/step
>5, d1[0.637] d2[0.466] g[56.407]
1/1 [==============================] - 0s 158ms/step
>6, d1[0.507] d2[0.400] g[50.602]
1/1 [==============================] - 0s 130ms/step
>7, d1[0.396] d2[0.388] g[63.855]
1/1 [==============================] - 0s 174ms/step
>8, d1[0.369] d2[0.381] g[54.989]
1/1 [==============================] - 0s 124ms/step
>9, d1[0.361] d2[0.374] g[48.804]
1/1 [==============================] - 0s 128ms/step
>10, d1[0.350] d2[0.363] g[56.872]
1/1 [==============================] - 0s 124ms/step
>11, d1[0.369] d2[0.378] g[34.839]
1/1 [==============================] - 0s 

ResourceExhaustedError: ignored

In [ ]:
name = "0070-GAN-0035-200"
save = r'/content/MyDrive'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)


In [ ]:
from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   '/images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 82ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 1000, n_validation = 20)

1/1 [==============================] - 0s 130ms/step
>1, d1[0.193] d2[0.266] g[8.188]
1/1 [==============================] - 0s 74ms/step
>2, d1[0.331] d2[0.493] g[9.777]
1/1 [==============================] - 0s 77ms/step
>3, d1[0.228] d2[0.165] g[9.450]
1/1 [==============================] - 0s 75ms/step
>4, d1[0.568] d2[0.286] g[6.705]
1/1 [==============================] - 0s 70ms/step
>5, d1[0.164] d2[0.304] g[7.984]
1/1 [==============================] - 0s 91ms/step
>6, d1[0.180] d2[0.210] g[7.388]
1/1 [==============================] - 0s 82ms/step
>7, d1[0.232] d2[0.146] g[7.085]
1/1 [==============================] - 0s 129ms/step
>8, d1[0.156] d2[0.170] g[6.963]
1/1 [==============================] - 0s 94ms/step
>9, d1[0.154] d2[0.307] g[9.357]
1/1 [==============================] - 0s 71ms/step
>10, d1[0.285] d2[0.172] g[7.991]
1/1 [==============================] - 0s 71ms/step
>11, d1[0.234] d2[0.730] g[7.010]
1/1 [==============================] - 0s 92ms/step
>12, d1[0

In [ ]:
name = "0071-GAN-0036-1200"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)


In [ ]:
from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 77ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

1/1 [==============================] - 0s 77ms/step
>1, d1[0.144] d2[0.177] g[8.610]
1/1 [==============================] - 0s 76ms/step
>2, d1[0.127] d2[0.249] g[13.981]
1/1 [==============================] - 0s 76ms/step
>3, d1[0.414] d2[0.171] g[6.830]
1/1 [==============================] - 0s 71ms/step
>4, d1[0.337] d2[0.382] g[6.778]
1/1 [==============================] - 0s 75ms/step
>5, d1[0.443] d2[0.273] g[5.228]
1/1 [==============================] - 0s 78ms/step
>6, d1[0.381] d2[0.356] g[4.419]
1/1 [==============================] - 0s 95ms/step
>7, d1[0.208] d2[0.335] g[7.073]
1/1 [==============================] - 0s 83ms/step
>8, d1[0.280] d2[0.257] g[6.416]
1/1 [==============================] - 0s 74ms/step
>9, d1[0.161] d2[0.327] g[8.779]
1/1 [==============================] - 0s 75ms/step
>10, d1[0.251] d2[0.519] g[7.474]
1/1 [==============================] - 0s 72ms/step
>11, d1[0.946] d2[0.188] g[6.675]
1/1 [==============================] - 0s 80ms/step
>12, d1[0.

In [ ]:
name = "0072-GAN-0037-1700"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)


In [ ]:
from tensorflow import keras
from tensorflow.keras.utils import save_img
name = "0072-GAN-0037-1700"
#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #111111

1/1 [==============================] - 0s 73ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

1/1 [==============================] - 0s 79ms/step
>1, d1[0.483] d2[0.269] g[4.988]
1/1 [==============================] - 0s 80ms/step
>2, d1[0.032] d2[0.265] g[8.225]
1/1 [==============================] - 0s 74ms/step
>3, d1[0.111] d2[0.189] g[8.593]
1/1 [==============================] - 0s 97ms/step
>4, d1[0.559] d2[0.323] g[3.695]
1/1 [==============================] - 0s 76ms/step
>5, d1[0.372] d2[0.309] g[4.712]
1/1 [==============================] - 0s 79ms/step
>6, d1[0.194] d2[0.239] g[6.641]
1/1 [==============================] - 0s 92ms/step
>7, d1[0.144] d2[0.200] g[6.360]
1/1 [==============================] - 0s 76ms/step
>8, d1[0.024] d2[0.176] g[10.297]
1/1 [==============================] - 0s 74ms/step
>9, d1[0.081] d2[0.132] g[7.401]
1/1 [==============================] - 0s 72ms/step
>10, d1[0.270] d2[0.178] g[6.710]
1/1 [==============================] - 0s 74ms/step
>11, d1[0.121] d2[0.239] g[6.354]
1/1 [==============================] - 0s 74ms/step
>12, d1[0.

In [ ]:
name = "0073-GAN-0038-2200"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)


In [ ]:
from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 95ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

1/1 [==============================] - 0s 204ms/step
>1, d1[0.329] d2[0.429] g[3.821]
1/1 [==============================] - 0s 73ms/step
>2, d1[0.292] d2[0.285] g[4.759]
1/1 [==============================] - 0s 78ms/step
>3, d1[0.233] d2[0.359] g[8.094]
1/1 [==============================] - 0s 89ms/step
>4, d1[0.114] d2[0.165] g[13.136]
1/1 [==============================] - 0s 80ms/step
>5, d1[0.186] d2[0.223] g[7.740]
1/1 [==============================] - 0s 77ms/step
>6, d1[0.113] d2[0.452] g[9.061]
1/1 [==============================] - 0s 73ms/step
>7, d1[0.676] d2[0.126] g[6.324]
1/1 [==============================] - 0s 76ms/step
>8, d1[0.638] d2[0.220] g[5.852]
1/1 [==============================] - 0s 74ms/step
>9, d1[0.042] d2[0.294] g[9.882]
1/1 [==============================] - 0s 72ms/step
>10, d1[0.043] d2[0.197] g[9.847]
1/1 [==============================] - 0s 71ms/step
>11, d1[0.708] d2[0.168] g[5.056]
1/1 [==============================] - 0s 75ms/step
>12, d1[0

In [ ]:
name = "0074-GAN-0039-2700"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

In [ ]:
from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 106ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

1/1 [==============================] - 0s 299ms/step
>1, d1[0.189] d2[0.311] g[6.462]
1/1 [==============================] - 0s 186ms/step
>2, d1[0.077] d2[0.196] g[7.104]
1/1 [==============================] - 0s 162ms/step
>3, d1[0.226] d2[0.244] g[6.425]
1/1 [==============================] - 0s 83ms/step
>4, d1[0.063] d2[0.258] g[7.856]
1/1 [==============================] - 0s 96ms/step
>5, d1[0.073] d2[0.166] g[8.218]
1/1 [==============================] - 0s 76ms/step
>6, d1[0.543] d2[0.394] g[3.564]
1/1 [==============================] - 0s 75ms/step
>7, d1[0.066] d2[0.182] g[8.528]
1/1 [==============================] - 0s 85ms/step
>8, d1[0.682] d2[0.175] g[5.531]
1/1 [==============================] - 0s 82ms/step
>9, d1[0.067] d2[0.272] g[7.409]
1/1 [==============================] - 0s 75ms/step
>10, d1[0.650] d2[0.196] g[4.770]
1/1 [==============================] - 0s 75ms/step
>11, d1[0.269] d2[0.311] g[5.608]
1/1 [==============================] - 0s 74ms/step
>12, d1[

In [ ]:
name = "0075-GAN-0040-3200"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

In [ ]:
from tensorflow import keras
from tensorflow.keras.utils import save_img
name = "0075-GAN-0040-3200"
#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 50ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

1/1 [==============================] - 0s 347ms/step
>1, d1[0.056] d2[0.411] g[8.529]
1/1 [==============================] - 0s 72ms/step
>2, d1[0.475] d2[0.168] g[6.062]
1/1 [==============================] - 0s 107ms/step
>3, d1[0.425] d2[0.324] g[6.879]
1/1 [==============================] - 0s 314ms/step
>4, d1[0.051] d2[0.230] g[10.869]
1/1 [==============================] - 0s 99ms/step
>5, d1[0.360] d2[0.371] g[6.121]
1/1 [==============================] - 0s 74ms/step
>6, d1[0.455] d2[0.314] g[6.392]
1/1 [==============================] - 0s 207ms/step
>7, d1[0.619] d2[0.283] g[6.424]
1/1 [==============================] - 0s 208ms/step
>8, d1[0.368] d2[0.558] g[6.937]
1/1 [==============================] - 0s 140ms/step
>9, d1[0.241] d2[0.342] g[7.560]
1/1 [==============================] - 0s 88ms/step
>10, d1[0.406] d2[0.252] g[6.306]
1/1 [==============================] - 0s 75ms/step
>11, d1[0.061] d2[0.350] g[10.272]
1/1 [==============================] - 0s 102ms/step
>1

In [ ]:
name = "0076-GAN-0041-3700"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 40ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

name = "0077-GAN-0042-4200"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 78ms/step
>1, d1[0.239] d2[0.186] g[6.211]
1/1 [==============================] - 0s 114ms/step
>2, d1[0.259] d2[0.777] g[2.994]
1/1 [==============================] - 0s 106ms/step
>3, d1[0.157] d2[0.208] g[4.791]
1/1 [==============================] - 0s 79ms/step
>4, d1[0.111] d2[0.150] g[6.699]
1/1 [==============================] - 0s 84ms/step
>5, d1[0.130] d2[0.125] g[8.056]
1/1 [==============================] - 0s 79ms/step
>6, d1[0.265] d2[0.384] g[5.715]
1/1 [==============================] - 0s 54ms/step
>7, d1[0.266] d2[0.252] g[5.518]
1/1 [==============================] - 0s 96ms/step
>8, d1[0.121] d2[0.142] g[6.138]
1/1 [==============================] - 0s 76ms/step
>9, d1[0.038] d2[0.117] g[10.590]
1/1 [==============================] - 0s 75ms/step
>10, d1[0.083] d2[0.159] g[8.384]
1/1 [==============================] - 0s 96ms/step
>11, d1[0.068] d2[0.126] g[6.867]
1/1 [==============================] - 0s 87ms/step
>12, d1[

1/1 [==============================] - 0s 45ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

name = "0078-GAN-0043-4700"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 137ms/step
>1, d1[0.631] d2[0.227] g[4.131]
1/1 [==============================] - 0s 79ms/step
>2, d1[0.093] d2[0.246] g[5.875]
1/1 [==============================] - 0s 65ms/step
>3, d1[0.296] d2[0.269] g[4.879]
1/1 [==============================] - 0s 79ms/step
>4, d1[0.116] d2[0.239] g[5.927]
1/1 [==============================] - 0s 78ms/step
>5, d1[0.029] d2[0.223] g[11.762]
1/1 [==============================] - 0s 78ms/step
>6, d1[0.523] d2[0.403] g[4.198]
1/1 [==============================] - 0s 77ms/step
>7, d1[0.276] d2[0.267] g[4.606]
1/1 [==============================] - 0s 75ms/step
>8, d1[0.062] d2[0.274] g[10.279]
1/1 [==============================] - 0s 75ms/step
>9, d1[0.113] d2[0.175] g[7.977]
1/1 [==============================] - 0s 77ms/step
>10, d1[0.603] d2[0.168] g[5.085]
1/1 [==============================] - 0s 76ms/step
>11, d1[0.349] d2[0.333] g[3.570]
1/1 [==============================] - 0s 77ms/step
>12, d1[

1/1 [==============================] - 0s 37ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

name = "0079-GAN-0044-5200"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 77ms/step
>1, d1[0.410] d2[0.208] g[5.633]
1/1 [==============================] - 0s 52ms/step
>2, d1[0.258] d2[0.218] g[5.683]
1/1 [==============================] - 0s 79ms/step
>3, d1[0.031] d2[0.232] g[6.008]
1/1 [==============================] - 0s 74ms/step
>4, d1[0.418] d2[0.689] g[4.404]
1/1 [==============================] - 0s 80ms/step
>5, d1[0.196] d2[0.287] g[3.550]
1/1 [==============================] - 0s 73ms/step
>6, d1[0.401] d2[0.405] g[4.994]
1/1 [==============================] - 0s 73ms/step
>7, d1[0.324] d2[0.270] g[6.270]
1/1 [==============================] - 0s 86ms/step
>8, d1[0.264] d2[0.248] g[6.718]
1/1 [==============================] - 0s 76ms/step
>9, d1[0.066] d2[0.276] g[6.665]
1/1 [==============================] - 0s 78ms/step
>10, d1[0.126] d2[0.255] g[6.470]
1/1 [==============================] - 0s 73ms/step
>11, d1[0.358] d2[0.291] g[7.115]
1/1 [==============================] - 0s 76ms/step
>12, d1[0.3

1/1 [==============================] - 0s 38ms/step


In [ ]:
for i in range(6): 
  a,b = next(preprocess(train_gen))
  p = g_model.predict(a)
  plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  plt.imshow(p[0])
  plt.show()
  plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  plt.imshow(b[0])
  plt.show()
  #save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

In [ ]:
for i in range(6): 
  a,b = next(preprocess(test_gen))
  p = g_model.predict(a)
  plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  plt.imshow(p[0])
  plt.show()
  plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  plt.imshow(b[0])
  plt.show()
  #save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

name = "0078-GAN-0043-4700"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

name = "0080-GAN-0045-5700"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 107ms/step
>1, d1[0.200] d2[0.120] g[6.074]
1/1 [==============================] - 0s 97ms/step
>2, d1[0.358] d2[0.238] g[6.156]
1/1 [==============================] - 0s 71ms/step
>3, d1[0.140] d2[0.284] g[6.844]
1/1 [==============================] - 0s 72ms/step
>4, d1[0.689] d2[0.266] g[4.821]
1/1 [==============================] - 0s 75ms/step
>5, d1[0.445] d2[0.402] g[4.141]
1/1 [==============================] - 0s 74ms/step
>6, d1[0.019] d2[0.199] g[12.261]
1/1 [==============================] - 0s 71ms/step
>7, d1[0.221] d2[0.410] g[7.145]
1/1 [==============================] - 0s 76ms/step
>8, d1[0.572] d2[0.249] g[5.065]
1/1 [==============================] - 0s 73ms/step
>9, d1[0.031] d2[0.503] g[8.755]
1/1 [==============================] - 0s 71ms/step
>10, d1[0.520] d2[0.292] g[7.322]
1/1 [==============================] - 0s 72ms/step
>11, d1[0.618] d2[0.292] g[5.576]
1/1 [==============================] - 0s 66ms/step
>12, d1[0

1/1 [==============================] - 0s 43ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

name = "0081-GAN-0046-6200"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 77ms/step
>1, d1[0.524] d2[0.326] g[5.195]
1/1 [==============================] - 0s 78ms/step
>2, d1[0.447] d2[0.352] g[4.953]
1/1 [==============================] - 0s 75ms/step
>3, d1[0.024] d2[0.234] g[8.544]
1/1 [==============================] - 0s 76ms/step
>4, d1[0.247] d2[0.185] g[4.991]
1/1 [==============================] - 0s 76ms/step
>5, d1[0.183] d2[0.264] g[6.018]
1/1 [==============================] - 0s 76ms/step
>6, d1[0.228] d2[0.370] g[5.499]
1/1 [==============================] - 0s 76ms/step
>7, d1[0.268] d2[0.402] g[4.904]
1/1 [==============================] - 0s 75ms/step
>8, d1[0.058] d2[0.225] g[7.205]
1/1 [==============================] - 0s 97ms/step
>9, d1[0.441] d2[0.337] g[4.440]
1/1 [==============================] - 0s 75ms/step
>10, d1[0.148] d2[0.242] g[3.673]
1/1 [==============================] - 0s 74ms/step
>11, d1[0.140] d2[0.333] g[4.471]
1/1 [==============================] - 0s 73ms/step
>12, d1[0.2

1/1 [==============================] - 0s 49ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

name = "0082-GAN-0047-6700"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 84ms/step
>1, d1[0.704] d2[0.323] g[4.158]
1/1 [==============================] - 0s 99ms/step
>2, d1[0.495] d2[0.463] g[6.468]
1/1 [==============================] - 0s 79ms/step
>3, d1[0.054] d2[0.303] g[8.272]
1/1 [==============================] - 0s 74ms/step
>4, d1[0.077] d2[0.200] g[10.018]
1/1 [==============================] - 0s 77ms/step
>5, d1[0.718] d2[0.168] g[7.749]
1/1 [==============================] - 0s 148ms/step
>6, d1[0.428] d2[0.394] g[7.895]
1/1 [==============================] - 0s 93ms/step
>7, d1[0.193] d2[0.241] g[8.311]
1/1 [==============================] - 0s 79ms/step
>8, d1[0.354] d2[0.306] g[7.263]
1/1 [==============================] - 0s 101ms/step
>9, d1[0.391] d2[0.193] g[5.290]
1/1 [==============================] - 0s 94ms/step
>10, d1[0.431] d2[0.443] g[2.847]
1/1 [==============================] - 0s 76ms/step
>11, d1[0.082] d2[0.323] g[7.298]
1/1 [==============================] - 0s 54ms/step
>12, d1[

1/1 [==============================] - 0s 42ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

name = "0083-GAN-0048-7200"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 78ms/step
>1, d1[0.174] d2[0.225] g[7.049]
1/1 [==============================] - 0s 94ms/step
>2, d1[0.100] d2[0.168] g[9.384]
1/1 [==============================] - 0s 188ms/step
>3, d1[0.103] d2[0.169] g[9.923]
1/1 [==============================] - 0s 77ms/step
>4, d1[0.683] d2[0.283] g[6.268]
1/1 [==============================] - 0s 75ms/step
>5, d1[0.335] d2[0.272] g[6.119]
1/1 [==============================] - 0s 74ms/step
>6, d1[0.030] d2[0.170] g[9.458]
1/1 [==============================] - 0s 84ms/step
>7, d1[0.562] d2[0.258] g[4.779]
1/1 [==============================] - 0s 76ms/step
>8, d1[0.612] d2[0.283] g[4.447]
1/1 [==============================] - 0s 75ms/step
>9, d1[0.384] d2[0.351] g[5.667]
1/1 [==============================] - 0s 52ms/step
>10, d1[0.339] d2[0.352] g[4.985]
1/1 [==============================] - 0s 76ms/step
>11, d1[0.135] d2[0.252] g[8.873]
1/1 [==============================] - 0s 74ms/step
>12, d1[0.

1/1 [==============================] - 0s 36ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

name = "0084-GAN-0049-7700"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

1/1 [==============================] - 0s 82ms/step
>1, d1[0.413] d2[0.376] g[4.360]
1/1 [==============================] - 0s 77ms/step
>2, d1[0.170] d2[0.284] g[6.126]
1/1 [==============================] - 0s 83ms/step
>3, d1[0.568] d2[0.205] g[5.776]
1/1 [==============================] - 0s 73ms/step


In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

name = "0085-GAN-0050-8200"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

name = "0086-GAN-0051-8700"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #

In [ ]:
history = train(d_model, g_model, gan_model, preprocess(train_gen), "", n_batch = 2,n_steps = 250, n_validation = 20)

name = "0087-GAN-0052-9200"
save = r'/content/drive/MyDrive/'
d_model.save(save + "d-"+ name, include_optimizer=True)
g_model.save(save + "g-"+ name, include_optimizer=True)
gan_model.save(save + "gan-" + name, include_optimizer=True)

from tensorflow import keras
from tensorflow.keras.utils import save_img

#name = "0042-GAN-0017-2700"
#save = r'/content/drive/MyDrive/Uni/Cuarto curso/Primer cuatrimestre/TFG/WebScrapping/'
#g_model = keras.models.load_model(save + "g-"+ name)

for i in range(6): 
  a,b = next(preprocess(mini_test_gen))
  p = g_model.predict(a)
  #plt.figure(figsize=(img_height/96, img_width/96), dpi=96)
  #plt.imshow(p[0])
  #plt.show()
  save_img(save +   'images2/' + name + '-' + str(i) + ".jpg", p[0])
  #